# Notebook Imports

In [1]:
import pandas as pd
import numpy as np

import requests
from pydantic import BaseModel, ValidationError
from typing import List
from datetime import datetime
import pydantic_core
import concurrent.futures
import json

# Scraping & Parsing Data

## Creating Classes for Parsing

In [ ]:
# Create classes for parsing
class Hero(BaseModel):
    code: str
    grade: str
    name: str
    job_cd: str
    attribute_cd: str

class Heroes(BaseModel):
    en: List[Hero]

class Player(BaseModel):
    nick_no: int
    world_code: str

class MyTeam(BaseModel):
    pick_order: int
    hero_code: str
    attack_damage: int
    receive_damage: float
    recovery: int
    mvp_point: int
    artifact: str
    equip: list
    respawn: int
    mvp: int
    kill_count: int
    attribute_cd: str
    job_cd: str

class TeamBettleInfoenemy(BaseModel):
    my_team: List[MyTeam]
    
class TeamBettleInfo(BaseModel):
    my_team: List[MyTeam]
      
class Battle(BaseModel):
    battleCompletedate: datetime
    teamBettleInfoenemy: str
    prebanListEnemy: str
    teamBettleInfo: str
    prebanList: str
    iswin: int

class ResultBody(BaseModel):
    nick_no: int
    battle_list: List[Battle]

class BattleListResponse(BaseModel):
    result_body: ResultBody

# Players list
class PlayerListResponse(BaseModel):
    result_body: List[Player]

# Hero list 
class HeroListResponse(BaseModel):
    result_body: List[Heroes]

NameError: name 'BaseModel' is not defined

## Collecting Players ID Numbers

In [ ]:
# Create a function that collects query parameters to follow up with a request on battles of top 100 players 
nick_no_list = []
world_code_list = []

def get_query_data():
    try:
        player_resp = requests.post(
            url = 'https://epic7.gg.onstove.com/gameApi/getWorldUserRankingDetail?lang=en&season_code=&world_code=all',
            headers={
                'Accept-Language': 'en-US',
                'Accept': 'application/json, text/plain, */*',
            },
        )
        if player_resp.ok:
            player_response = PlayerListResponse.model_validate(player_resp.json())
            for i in range(len(player_response.result_body)):
                nick_no_list.append(player_response.result_body[i].nick_no)
                world_code_list.append(player_response.result_body[i].world_code)
            query_df = pd.DataFrame({'nick_no': nick_no_list, 'world_code': world_code_list})
            return query_df
        
        else:
            player_resp.raise_for_status()

    except Exception as e:
        print(e)
    

In [ ]:
get_query_data()

,nick_no,world_code
0,119456895,world_kor
1,192119856,world_eu
2,85251058,world_global
3,72428245,world_asia
4,71212252,world_asia
...,...,...
95,182147370,world_eu
96,200148716,world_global
97,60502181,world_global
98,63047239,world_global


## Collecting Battle Statistics

In [ ]:
# Create a function that requests and collects battle info of each player

#right side lists
right_pick_order = []
right_hero_code = []
right_attack_damage = []
right_receive_damage = []
right_recovery = []
right_mvp_point = []
right_artifact = []
right_equip = []
right_respawn= []
right_mvp= []
right_kill_count= []
right_attribute_cd= []
right_job_cd= []
right_postban = []
    
#left side lists
left_pick_order = []
left_hero_code = []
left_attack_damage = []
left_receive_damage = []
left_recovery = []
left_mvp_point = []
left_artifact = []
left_equip = []
left_respawn= []
left_mvp= []
left_kill_count= []
left_attribute_cd= []
left_job_cd= []
left_postban = []
    
right_preban = []
is_win = []
left_preban = []
first_pick = []

def get_player_battles():
#    try:
    #print(len(nick_no_list))
    for i in range(len(nick_no_list)):
    #for i in range(len(nick_no_list[:4])):
        #print(nick_no_list[i])
        resp = requests.post(
            url=f'https://epic7.gg.onstove.com/gameApi/getBattleList?nick_no={nick_no_list[i]}&world_code={world_code_list[i]}&lang=en&season_code=',
            headers={'Accept-Language': 'en-US', 'Accept': 'application/json, text/plain, */*'})
        #print(resp)
        if resp.ok:
            time.sleep(1)
            response = BattleListResponse.model_validate(resp.json())
            #print(response.result_body.battle_list)
            row_count = 0 # looking for rows with error in first pick for loop
            for j in range(len(response.result_body.battle_list)):
                response.result_body.battle_list[j].teamBettleInfoenemy = (  #preprocessing  data on right side characters and battle stats
                    TeamBettleInfoenemy.model_validate_json("{" + response.result_body.battle_list[j].teamBettleInfoenemy + "}"))
                for f in range(len(response.result_body.battle_list[j].teamBettleInfoenemy.my_team)): #gathering right side characters and battle stats
                    #print(len(response.result_body.battle_list[j].teamBettleInfoenemy.my_team))
                    #print(response.result_body.battle_list[j].teamBettleInfoenemy.my_team[f])
                    right_pick_order.append(response.result_body.battle_list[j].teamBettleInfoenemy.my_team[f].pick_order)
                    right_hero_code.append(response.result_body.battle_list[j].teamBettleInfoenemy.my_team[f].hero_code)
                    right_attack_damage.append(response.result_body.battle_list[j].teamBettleInfoenemy.my_team[f].attack_damage)
                    right_receive_damage.append(response.result_body.battle_list[j].teamBettleInfoenemy.my_team[f].receive_damage)
                    right_recovery.append(response.result_body.battle_list[j].teamBettleInfoenemy.my_team[f].recovery)
                    right_mvp_point.append(response.result_body.battle_list[j].teamBettleInfoenemy.my_team[f].mvp_point)
                    right_artifact.append(response.result_body.battle_list[j].teamBettleInfoenemy.my_team[f].artifact)
                    right_equip.append(response.result_body.battle_list[j].teamBettleInfoenemy.my_team[f].equip)
                    right_respawn.append(response.result_body.battle_list[j].teamBettleInfoenemy.my_team[f].respawn)
                    right_mvp.append(response.result_body.battle_list[j].teamBettleInfoenemy.my_team[f].mvp)
                    right_kill_count.append(response.result_body.battle_list[j].teamBettleInfoenemy.my_team[f].kill_count)
                    right_attribute_cd.append(response.result_body.battle_list[j].teamBettleInfoenemy.my_team[f].attribute_cd)
                    right_job_cd.append(response.result_body.battle_list[j].teamBettleInfoenemy.my_team[f].job_cd)
                    
                # preprocessing  data on right side bans and adding to the list
                right_side_preban = resp.json()['result_body']['battle_list'][j]['prebanListEnemy']
                json_string = f'{{ {right_side_preban} }}'
                right_side_preban_data = json.loads(json_string)
                right_preban.append(right_side_preban_data['preban_list'])
                
                for postban in range(len(resp.json()['result_body']['battle_list'][j]['enemy_deck']['hero_list'])):
                    if resp.json()['result_body']['battle_list'][j]['enemy_deck']['hero_list'][postban]['ban'] == 1:
                        is_right_postban = resp.json()['result_body']['battle_list'][j]['enemy_deck']['hero_list'][postban]['hero_code']
                        right_postban.append(is_right_postban)
                    # elif resp.json()['result_body']['battle_list'][j]['enemy_deck']['hero_list'][postban]['ban'] == 0:
                    #     pass
                    # else:
                    #     right_postban.append('no_postban_data')
        
                                
                response.result_body.battle_list[j].teamBettleInfo = (  #preprocessing  data on left side characters and battle stats
                    TeamBettleInfo.model_validate_json("{" + response.result_body.battle_list[j].teamBettleInfo + "}"))
                # print(len(response.result_body.battle_list[j].teamBettleInfo.my_team))
                for f in range(len(response.result_body.battle_list[j].teamBettleInfo.my_team)): #gathering left side characters and battle stats
                    left_pick_order.append(response.result_body.battle_list[j].teamBettleInfo.my_team[f].pick_order)
                    left_hero_code.append(response.result_body.battle_list[j].teamBettleInfo.my_team[f].hero_code)
                    left_attack_damage.append(response.result_body.battle_list[j].teamBettleInfo.my_team[f].attack_damage)
                    left_receive_damage.append(response.result_body.battle_list[j].teamBettleInfo.my_team[f].receive_damage)
                    left_recovery.append(response.result_body.battle_list[j].teamBettleInfo.my_team[f].recovery)
                    left_mvp_point.append(response.result_body.battle_list[j].teamBettleInfo.my_team[f].mvp_point)
                    left_artifact.append(response.result_body.battle_list[j].teamBettleInfo.my_team[f].artifact)
                    left_equip.append(response.result_body.battle_list[j].teamBettleInfo.my_team[f].equip)
                    left_respawn.append(response.result_body.battle_list[j].teamBettleInfo.my_team[f].respawn)
                    left_mvp.append(response.result_body.battle_list[j].teamBettleInfo.my_team[f].mvp)
                    left_kill_count.append(response.result_body.battle_list[j].teamBettleInfo.my_team[f].kill_count)
                    left_attribute_cd.append(response.result_body.battle_list[j].teamBettleInfo.my_team[f].attribute_cd)
                    left_job_cd.append(response.result_body.battle_list[j].teamBettleInfo.my_team[f].job_cd)
                                    
                #preprocessing  data on left side bans and adding to the list
                left_side_preban = resp.json()['result_body']['battle_list'][j]['prebanList']
                json_string = f'{{ {left_side_preban} }}'
                left_side_preban_data = json.loads(json_string)
                left_preban.append(left_side_preban_data['preban_list'])
                #print(resp.json()['result_body']['battle_list'][j]['my_deck']['hero_list'])
                for postban in range(len(resp.json()['result_body']['battle_list'][j]['my_deck']['hero_list'])):
                    print(resp.json()['result_body']['battle_list'][j]['my_deck']['hero_list'][postban])
                    if resp.json()['result_body']['battle_list'][j]['my_deck']['hero_list'][postban]['ban'] == 1:
                        is_left_postban = resp.json()['result_body']['battle_list'][j]['my_deck']['hero_list'][postban]['hero_code']
                        left_postban.append(is_left_postban)
                    # elif resp.json()['result_body']['battle_list'][j]['my_deck']['hero_list'][postban]['ban'] == 0:
                    #     print('Das ist chero')
                    # else:
                    #     left_postban.append('no_postban_data')
                        
                        
                #preprocessing  data on batte outcome and adding to the list
                right_side_win = resp.json()['result_body']['battle_list'][j]
                is_win.append(right_side_win['iswin'])
                #first pick
                #print(resp.json()['result_body']['battle_list'][j]['my_deck']['hero_list'])
                
                try:
                    left_first_pick = resp.json()['result_body']['battle_list'][j]['my_deck']['hero_list'][0]
                    first_pick.append(left_first_pick['first_pick'])
                    row_count +=1
                except IndexError: 
                    first_pick.append('no_fp_data')
                    left_postban.append('no_postban_data')
                    right_postban.append('no_postban_data')
                    print(f'No data on first pick in row {row_count}')
                
        
            
        else:
            resp.raise_for_status()
# except Exception as e:

#        print(e)

In [ ]:
get_player_battles()

{'hero_code': 'c1159', 'first_pick': 1, 'mvp': 0, 'ban': 0}
{'hero_code': 'c2039', 'first_pick': 0, 'mvp': 0, 'ban': 0}
{'hero_code': 'c6037', 'first_pick': 0, 'mvp': 0, 'ban': 0}
{'hero_code': 'c1096', 'first_pick': 0, 'mvp': 0, 'ban': 1}
{'hero_code': 'c1038', 'first_pick': 0, 'mvp': 1, 'ban': 0}
{'hero_code': 'c2112', 'first_pick': 0, 'mvp': 0, 'ban': 0}
{'hero_code': 'c2042', 'first_pick': 0, 'mvp': 0, 'ban': 1}
{'hero_code': 'c1019', 'first_pick': 0, 'mvp': 1, 'ban': 0}
{'hero_code': 'c5016', 'first_pick': 0, 'mvp': 0, 'ban': 0}
{'hero_code': 'c1151', 'first_pick': 0, 'mvp': 0, 'ban': 0}
{'hero_code': 'c2112', 'first_pick': 1, 'mvp': 0, 'ban': 1}
{'hero_code': 'c2039', 'first_pick': 0, 'mvp': 0, 'ban': 0}
{'hero_code': 'c1129', 'first_pick': 0, 'mvp': 0, 'ban': 0}
{'hero_code': 'c2069', 'first_pick': 0, 'mvp': 1, 'ban': 0}
{'hero_code': 'c2102', 'first_pick': 0, 'mvp': 0, 'ban': 0}
{'hero_code': 'c1159', 'first_pick': 1, 'mvp': 0, 'ban': 0}
{'hero_code': 'c2039', 'first_pick': 0, 

In [ ]:
# Checking the length of the lists prior to concatinating,
# since the scraped data was inconsistent and had missing values for variables first_pick,
# left_postban and right_postban.
print(len(first_pick), len(left_preban), len(right_preban),len(left_postban), len(right_postban),
      
      len(right_pick_order), len(right_hero_code), len(right_attack_damage), len(right_receive_damage),
      len(right_recovery), len(right_mvp_point), len(right_artifact), len(right_equip), len(right_respawn),
      len(right_mvp), len(right_kill_count), len(right_attribute_cd), len(right_job_cd),
      
      len(right_postban),
      
      len(left_pick_order), len(left_hero_code), len(left_attack_damage),
      len(left_receive_damage), len(left_recovery), len(left_mvp_point), len(left_artifact), len(left_equip),
      len(left_respawn), len(left_mvp), len(left_kill_count), len(left_attribute_cd), len(left_job_cd),
      
      len(left_postban), len(right_preban), len(is_win), len(left_preban), len(first_pick))

10000 10000 10000 10000 10000 49730 49730 49730 49730 49730 49730 49730 49730 49730 49730 49730 49730 49730 10000 49730 49730 49730 49730 49730 49730 49730 49730 49730 49730 49730 49730 49730 10000 10000 10000 10000 10000


## Concatinating Gathered Data

In [ ]:
def make_df():
# Right side df
    right_side_df = pd.DataFrame({'right_pick_order': right_pick_order, 'right_hero_code': right_hero_code, 'right_attack_damage': right_attack_damage, 
    'right_receive_damage': right_receive_damage, 'right_recovery': right_recovery, 'right_mvp_point': right_mvp_point,
    'right_artifact': right_artifact, 'right_equip': right_equip, 'right_respawn': right_respawn, 'right_mvp': right_mvp, 
    'right_kill_count': right_kill_count, 'right_attribute_cd': right_attribute_cd, 'right_job_cd': right_job_cd})
            
# Left side df
    left_side_df = pd.DataFrame({'left_pick_order': left_pick_order, 'left_hero_code': left_hero_code, 'left_attack_damage': left_attack_damage, 
    'left_receive_damage': left_receive_damage, 'left_recovery': left_recovery, 'left_mvp_point': left_mvp_point,
    'left_artifact': left_artifact, 'left_equip': left_equip, 'left_respawn': left_respawn, 'left_mvp': left_mvp, 
    'left_kill_count': left_kill_count, 'left_attribute_cd': left_attribute_cd, 'left_job_cd': left_job_cd})
    
    df_1 = pd.concat([left_side_df, right_side_df], axis= 1)
    
    return df_1
    
# Post bans and first pick have different lenght and will be collected in a separate df
df_2 = pd.DataFrame({'left_preban': left_preban, 'left_postban': left_postban,
                     'right_preban': right_preban, 'right_postban': right_postban,
                     'first_pick': first_pick, 'is_win': is_win})
    


In [ ]:
df_1 = make_df()
make_df()

,left_pick_order,left_hero_code,left_attack_damage,left_receive_damage,left_recovery,left_mvp_point,left_artifact,left_equip,left_respawn,left_mvp,...,right_receive_damage,right_recovery,right_mvp_point,right_artifact,right_equip,right_respawn,right_mvp,right_kill_count,right_attribute_cd,right_job_cd
0,1,c1159,0,0.00,0,8546,efw35,"[set_speed, set_max_hp]",0,0,...,27562.74,0,27562,efh14,"[set_acc, set_speed]",0,0,0,dark,manauser
1,2,c2039,1267,29297.90,4766,35331,efh06,"[set_counter, set_max_hp]",0,0,...,0.00,0,0,,[],0,0,0,light,knight
2,3,c6037,0,0.00,0,64096,efh20,"[set_immune, set_speed]",0,0,...,21514.00,128,36505,efa02,"[set_immune, set_speed]",0,0,0,fire,assassin
3,4,c1096,0,0.00,0,0,,[],0,0,...,70482.26,0,70482,ef507,"[set_res, set_speed]",0,0,0,dark,warrior
4,5,c1038,50131,0.00,0,67223,efa07,"[set_cri, set_att]",0,1,...,19075.44,2478,118116,ef504,"[set_speed, set_cri]",0,1,0,light,assassin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49725,1,c2090,0,0.00,0,40181,efh04,"[set_speed, set_max_hp]",0,0,...,0.00,0,22495,efm03,"[set_speed, set_immune]",0,0,0,light,mage
49726,2,c2042,0,9581.75,0,29672,efk03,"[set_max_hp, set_shield]",0,0,...,0.00,0,31004,efa14,"[set_cri_dmg, set_penetrate]",0,1,0,wind,assassin
49727,3,c1156,0,37581.07,0,37581,efk21,"[set_max_hp, set_speed]",0,0,...,0.00,0,24744,efh12,"[set_speed, set_res]",0,0,0,light,manauser
49728,4,c1151,0,0.00,0,0,,[],0,0,...,0.00,0,0,,[],0,0,0,light,mage


## Saving to Pickle

In [ ]:
# Save df in .pickle
df_1.to_pickle('./saved_files/data_1.pickle')
df_2.to_pickle('./saved_files/data_2.pickle')

In [3]:
df_read_1 =pd.read_pickle('./saved_files/data_1.pickle').copy() #.copy to keep the original df
df_read_2 =pd.read_pickle('./saved_files/data_2.pickle').copy()

In [4]:
df_read_1.tail()

,left_pick_order,left_hero_code,left_attack_damage,left_receive_damage,left_recovery,left_mvp_point,left_artifact,left_equip,left_respawn,left_mvp,...,right_receive_damage,right_recovery,right_mvp_point,right_artifact,right_equip,right_respawn,right_mvp,right_kill_count,right_attribute_cd,right_job_cd
49725,1,c2090,0,0.00,0,40181,efh04,"[set_speed, set_max_hp]",0,0,...,0.0,0,22495,efm03,"[set_speed, set_immune]",0,0,0,light,mage
49726,2,c2042,0,9581.75,0,29672,efk03,"[set_max_hp, set_shield]",0,0,...,0.0,0,31004,efa14,"[set_cri_dmg, set_penetrate]",0,1,0,wind,assassin
49727,3,c1156,0,37581.07,0,37581,efk21,"[set_max_hp, set_speed]",0,0,...,0.0,0,24744,efh12,"[set_speed, set_res]",0,0,0,light,manauser
49728,4,c1151,0,0.00,0,0,,[],0,0,...,0.0,0,0,,[],0,0,0,light,mage
49729,5,c2016,0,78403.23,0,83425,efk02,"[set_vampire, set_immune]",0,1,...,0.0,0,4988,efw01,"[set_scar, set_cri]",0,0,0,fire,warrior


In [5]:
df_read_1['left_pick_order'].value_counts()

left_pick_order
1    9946
2    9946
3    9946
4    9946
5    9946
Name: count, dtype: int64

In [6]:
df_read_1['right_pick_order'].value_counts()

right_pick_order
1    9946
2    9946
3    9946
4    9946
5    9946
Name: count, dtype: int64

## Reshaping the Dataframe 

In [7]:
# Create empty df with labels to be used in ML
df_reshaped = pd.DataFrame(columns= ['left_pick_order_1', 'left_pick_order_2', 'left_pick_order_3', 'left_pick_order_4', 'left_pick_order_5',                           #left
                                     'left_hero_code_1', 'left_hero_code_2', 'left_hero_code_3', 'left_hero_code_4', 'left_hero_code_5',
                                     'left_attack_damage_1', 'left_attack_damage_2', 'left_attack_damage_3', 'left_attack_damage_4', 'left_attack_damage_5',
                                    'left_receive_damage_1', 'left_receive_damage_2', 'left_receive_damage_3', 'left_receive_damage_4', 'left_receive_damage_5',
                                    'left_kill_count_1', 'left_kill_count_2', 'left_kill_count_3', 'left_kill_count_4', 'left_kill_count_5',
                                    'left_attribute_cd_1', 'left_attribute_cd_2', 'left_attribute_cd_3', 'left_attribute_cd_4', 'left_attribute_cd_5',
                                    'left_job_cd_1', 'left_job_cd_2', 'left_job_cd_3', 'left_job_cd_4', 'left_job_cd_5',
                                    'left_preban', 'left_postban',
                                    'right_pick_order_1', 'right_pick_order_2', 'right_pick_order_3', 'right_pick_order_4', 'right_pick_order_5',                          #right 
                                    'right_hero_code_1', 'right_hero_code_2', 'right_hero_code_3', 'right_hero_code_4', 'right_hero_code_5',
                                    'right_attack_damage_1', 'right_attack_damage_2', 'right_attack_damage_3', 'right_attack_damage_4', 'right_attack_damage_5',
                                    'right_receive_damage_1', 'right_receive_damage_2', 'right_receive_damage_3', 'right_receive_damage_4', 'right_receive_damage_5',
                                    'right_kill_count_1', 'right_kill_count_2', 'right_kill_count_3', 'right_kill_count_4', 'right_kill_count_5',
                                    'right_attribute_cd_1', 'right_attribute_cd_2', 'right_attribute_cd_3', 'right_attribute_cd_4', 'right_attribute_cd_5',
                                    'right_job_cd_1', 'right_job_cd_2', 'right_job_cd_3', 'right_job_cd_4', 'right_job_cd_5', 
                                    'right_preban', 'right_postban', 'first_pick', 'is_win'], 
                           index= np.arange(0, 10000))
    
df_reshaped.head()


,left_pick_order_1,left_pick_order_2,left_pick_order_3,left_pick_order_4,left_pick_order_5,left_hero_code_1,left_hero_code_2,left_hero_code_3,left_hero_code_4,left_hero_code_5,...,right_attribute_cd_5,right_job_cd_1,right_job_cd_2,right_job_cd_3,right_job_cd_4,right_job_cd_5,right_preban,right_postban,first_pick,is_win
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Fill in the empty df with gathered data
counter = 0
pick_order_counter = 1
for index, row in df_read_1.iterrows():
    if pick_order_counter == 6:
        pick_order_counter = 1
        counter +=1
    df_reshaped.iloc[counter][f'left_pick_order_{pick_order_counter}'] = row['left_pick_order']
    df_reshaped.iloc[counter][f'left_hero_code_{pick_order_counter}'] = row['left_hero_code']
    df_reshaped.iloc[counter][f'left_attack_damage_{pick_order_counter}'] = row['left_attack_damage']
    df_reshaped.iloc[counter][f'left_receive_damage_{pick_order_counter}'] = row['left_receive_damage']
    df_reshaped.iloc[counter][f'left_kill_count_{pick_order_counter}'] = row['left_kill_count']
    df_reshaped.iloc[counter][f'left_attribute_cd_{pick_order_counter}'] = row['left_attribute_cd']
    df_reshaped.iloc[counter][f'left_job_cd_{pick_order_counter}'] = row['left_job_cd']
        
    df_reshaped.iloc[counter][f'right_pick_order_{pick_order_counter}'] = row['right_pick_order']
    df_reshaped.iloc[counter][f'right_hero_code_{pick_order_counter}'] = row['right_hero_code']
    df_reshaped.iloc[counter][f'right_attack_damage_{pick_order_counter}'] = row['right_attack_damage']
    df_reshaped.iloc[counter][f'right_receive_damage_{pick_order_counter}'] = row['right_receive_damage']
    df_reshaped.iloc[counter][f'right_kill_count_{pick_order_counter}'] = row['right_kill_count']
    df_reshaped.iloc[counter][f'right_attribute_cd_{pick_order_counter}'] = row['right_attribute_cd']
    df_reshaped.iloc[counter][f'right_job_cd_{pick_order_counter}'] = row['right_job_cd']
    
    pick_order_counter +=1    
    
    
df_reshaped['left_preban'] = df_read_2['left_preban']
df_reshaped['left_postban'] = df_read_2['left_postban']

    
df_reshaped['right_preban'] = df_read_2['right_preban']
df_reshaped['right_postban'] = df_read_2['right_postban']

df_reshaped['first_pick'] = df_read_2['first_pick']
df_reshaped['is_win'] = df_read_2['is_win']  

    
    

/tmp/ipykernel_23527/673397446.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_reshaped.iloc[counter][f'left_pick_order_{pick_order_counter}'] = row['left_pick_order']
/tmp/ipykernel_23527/673397446.py:9: FutureWarning: ChainedAssignm

In [9]:
df_reshaped

,left_pick_order_1,left_pick_order_2,left_pick_order_3,left_pick_order_4,left_pick_order_5,left_hero_code_1,left_hero_code_2,left_hero_code_3,left_hero_code_4,left_hero_code_5,...,right_attribute_cd_5,right_job_cd_1,right_job_cd_2,right_job_cd_3,right_job_cd_4,right_job_cd_5,right_preban,right_postban,first_pick,is_win
0,1,2,3,4,5,c1159,c2039,c6037,c1096,c1038,...,light,manauser,knight,assassin,warrior,assassin,"[c2112, c2066]",c2042,1,1
1,1,2,3,4,5,c2112,c2042,c1019,c5016,c1151,...,light,knight,warrior,assassin,manauser,knight,"[c2039, c1055]",c2090,0,1
2,1,2,3,4,5,c2112,c2039,c1129,c2069,c2102,...,wind,knight,assassin,assassin,mage,assassin,"[c1118, c2066]",c1014,1,1
3,1,2,3,4,5,c1159,c2039,c6037,c1096,c1135,...,ice,manauser,knight,warrior,assassin,mage,"[c2112, c1125]",c1103,1,1
4,1,2,3,4,5,c1159,c2042,c6037,c1135,c1096,...,ice,manauser,manauser,knight,warrior,mage,"[c1151, c2112]",c2022,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"[c2112, c2090]",c1134,1,1
9996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"[c2039, c1133]",c1055,0,2
9997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"[c1118, c1151]",c1103,0,2
9998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"[c1118, c1133]",c1055,1,1


In [10]:
# Save reshaped dataframe
df_reshaped.to_pickle('./saved_files/df_reshaped.pickle')

In [ ]:
# Collect hero codes and names

hero_names = []
hero_codes = []    

url = "https://static.smilegatemegaport.com/gameRecord/epic7/epic7_hero.json?_=1721065508349"

# Send a GET request to fetch the JSON data
response = requests.get(url)


# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON data
    data = response.json()
    
# Iterate through each hero entry in the JSON data
    for hero in data['en']:
        hero_names.append(hero['name'])     
        hero_codes.append(hero['code'])     

heroes_df = pd.DataFrame(index = hero_codes, data = hero_names)

In [ ]:
# Save hero info 
heroes_df.to_pickle('./saved_files/heroes_names.pickle')